In [ ]:
import json
import pickle
from tqdm import tqdm
from constants import *
from sknetwork.ranking import PageRank, HITS
from sknetwork.data import from_edge_list

In [ ]:
reference_network = dict()
with open(PAPER_DATA_PATH, 'r') as f:
    for i, line in enumerate(tqdm(f, total=TOTAL_PAPER_COUNT+2)):
        if i == 0 or i == TOTAL_PAPER_COUNT+1:
            continue
        if i == 1:
            doc = json.loads(line)
        else:
            doc = json.loads(line[1:])
        
        id_int = int(doc['id'])
        if id_int not in reference_network:
            reference_network[id_int] = list()
        # print(doc.keys())
        reference_network[id_int] = reference_network[id_int] + doc.get('references', [])
                

In [ ]:
dataset_json = list()
for key in reference_network:
    dataset_json.append({'cited_id': key, 'citing_ids': reference_network[key]})

print(len(dataset_json))
json.dump(dataset_json, open(f'{SCRACTCH_PATH}/reference_network.json', 'wt+'))

In [6]:
# !pip install scikit-network
reference_network = json.load(open(f'{SCRACTCH_PATH}/reference_network.json', 'rt'))

In [8]:
edgelist = []
for a in reference_network:
    key = a['cited_id']
    for n in a['citing_ids']: edgelist.append((key, n)) 

print(len(edgelist))
graph = from_edge_list(edgelist)

45564149


In [9]:
pgr = PageRank()
pg_scores = (pgr.fit(graph.adjacency)).scores_.tolist()
hits = HITS()
scores = hits.fit(graph.adjacency)
hub_scores = scores.scores_row_.tolist()
auth_scores = scores.scores_col_.tolist()

In [11]:
from sknetwork.clustering import Louvain
louvain = Louvain()
membership_mat = louvain.fit_transform(graph.adjacency)

In [12]:
from sknetwork.hierarchy import Paris
paris = Paris()
dendrogram = paris.fit_predict(graph.adjacency)

In [14]:
from sknetwork.embedding import PCA
pca = PCA(3)
pca_scores = pca.fit_transform(graph.adjacency)

In [ ]:
data = {
    'node_names': graph.names,
    'pgr_scores': pg_scores,
    'hub_scores': hub_scores,
    'auth_scores': auth_scores,
    'paris_hierarchy': dendrogram,
    'louvain_membership': membership_mat,
    'pca_scores': pca_scores
}
pickle.dump(data, open(f'{SCRACTCH_PATH}/various_metrics.pickle', 'wb'))

membership_mat.shape